In [1]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle as plk
import importlib.util
import struct

###############################################################
# LINUX PATH

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/trainner/Documentos/Vh/photonics-ring-resonator')) #Current directory

###############################################################
# WINDOWS PATH

#spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
#lumapi = importlib.util.module_from_spec(spec_win)
#os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
#sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/Microring resonator/LumericalFiles/'))
#spec_win.loader.exec_module(lumapi)

In [2]:
lumR = lumapi.FDTD(filename='anel_lumerical.fsp',hide=False) 

In [3]:

def DeletarObj(nome, lum):
    lum.select(nome)
    lum.delete()


def CriarRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum):
    
    lum.addrect()
    lum.set({
        "name": nome,
        "x min": Xmin,"y": Y,"z min": Zmin,
        "x max": Xmax, "y span": Yspan, "z max": Zmax,
        "material": material
        })

def altRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum):
    DeletarObj(nome, lum)
    CriarRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum)

def altRect2(nome, X, Xspan, Y, Yspan, Z, Zspan, material, lum):
    Xmin = X - (Xspan/2)
    Xmax = X + (Xspan/2)
    Zmin = Z - (Zspan/2)
    Zmax = Z + (Zspan/2)
    altRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum)




In [4]:
RingLen = 0 # Length of straight sections of the ring
Radius = 3e-6 # Ring radius

RingSpan = 10e-6 # Length of straight waveguides
Gap = 150e-9 # Gap between waveguides and the ring

WGHeight = 0.22e-6 # height of the waveguide
WGWidth = 0.5e-6 # width of the waveguide base

FPoints = 8        # Monitor frequency points

In [10]:
# lumR.switchtolayout()
# lumR.selectall()
# lumR.delete()
DeletarObj("ring resonator", lumR)
lumR.addobject("ring_resonator")
lumR.set({"x":-RingSpan/2, "y":0, "z":0,
          "Lc":RingLen,
          "x span":RingSpan,
          "radius":Radius,
          "gap":Gap,
          "base height":WGHeight,
          "base width":WGWidth})

lumR.setnamed("ring resonator::inner_bottom","enabled",0)
lumR.setnamed("ring resonator::outer_bottom","enabled",0)
lumR.setnamed("ring resonator::arc2","enabled",0)
lumR.setnamed("ring resonator::arc3","enabled",0)

altRect2("leg1", -RingLen-Radius, WGWidth, -Radius/4, Radius/2, 0, WGHeight, "Si (Silicon) - Palik", lumR)
altRect2("leg2", RingLen+Radius, WGWidth, -Radius/4, Radius/2, 0, WGHeight, "Si (Silicon) - Palik", lumR)

DeletarObj("FDTD", lumR)
lumR.addfdtd()
lumR.set({"x":0, "y":0.5*(Radius+3*WGWidth+Gap - Radius/4), "z":0,
          "x span":RingSpan, "y span":Radius+3*WGWidth+Gap+Radius/2, "z span":3*WGWidth,
          "background material": "SiO2 (Glass) - Palik",
          "mesh accuracy":4})

lumR.addport()
lumR.set({"x":-RingSpan/2.5, "y":Radius+WGWidth+Gap, "z":0,
          "y span":4*WGWidth, "z span":2*WGWidth})
lumR.addport()
lumR.set({"x":RingSpan/2.5, "y":Radius+WGWidth+Gap, "z":0,
          "y span":4*WGWidth, "z span":2*WGWidth,
          "direction":'Backward'})
lumR.addport()
lumR.set({"x":-RingLen-Radius, "y":-Radius/4, "z":0,
          "injection axis":'y-axis', "x span":4*WGWidth, "z span":2*WGWidth})
lumR.addport()
lumR.set({"x":RingLen+Radius, "y":-Radius/4, "z":0,
          "injection axis":'y-axis', "x span":4*WGWidth, "z span":2*WGWidth})

lumR.select('FDTD::ports')
lumR.set('monitor frequency points',FPoints)

lumR.setglobalsource("wavelength start", 1.5e-6)
lumR.setglobalsource("wavelength stop", 1.6e-6)

1.0

In [12]:
# Sweep configs
FPoints = 8        # Monitor frequency points
SPoints = 5       # Sweep points
SRange = [2,6]  # Sweep range
Name = "Mesh_sweep"

# Sweep creation
lumR.deletesweep(Name)
lumR.addsweep(0)
lumR.setsweep("sweep","name",Name)
lumR.setsweep(Name,"type","Ranges")
lumR.setsweep(Name,"number of points",SPoints)

MeshParameter = {"Name":"Mesh accuracy", "Type":"Number",
               "Parameter":"::model::FDTD::mesh accuracy",
               "Start":SRange[0], "Stop":SRange[1]}

Transmission_In = {"Name":"Transmission In",
                   "Result":"::model::FDTD::ports::port 1::T"}
Transmission_Pass = {"Name":"Transmission Pass",
                     "Result":"::model::FDTD::ports::port 2::T"}
Transmission_Drop = {"Name":"Transmission Drop",
                     "Result":"::model::FDTD::ports::port 4::T"}

lumR.addsweepparameter(Name,MeshParameter)
lumR.addsweepresult(Name,Transmission_In)
lumR.addsweepresult(Name,Transmission_Pass)
lumR.addsweepresult(Name,Transmission_Drop)

lumR.select('FDTD::ports')
lumR.set('monitor frequency points',FPoints)

In [13]:
print(FPoints)

8


In [ ]:
plt.plot()